In [4]:
import os
import shutil
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/dogvscat"

# 源数据目录和目标分类目录
source_dir = '/content/drive/My Drive/dogvscat/images'
binary_dir = '/content/drive/My Drive/dogvscat/binary_classification'
multiclass_dir = '/content/drive/My Drive/dogvscat/multiclass_classification'

# 创建二分类和多分类的目录
os.makedirs(binary_dir, exist_ok=True)
os.makedirs(multiclass_dir, exist_ok=True)

# 创建猫和狗的分类目录
os.makedirs(os.path.join(binary_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(binary_dir, 'Dog'), exist_ok=True)



#classify types
for filename in os.listdir(source_dir):
    if filename.endswith('.jpg'):
        # 提取品种名称：去除编号和.jpg，通常编号前有一个下划线
        breed = '_'.join(filename.split('_')[:-1])  # 获取除最后一个元素外的所有元素并用下划线连接
        source_file = os.path.join(source_dir, filename)

        # 根据品种名称的首字母判断是猫还是狗
        if breed[0].isupper():  # 大写字母开头为猫
            shutil.copy(source_file, os.path.join(binary_dir, 'Cat', filename))
            # 同时复制到多分类目录中的对应品种文件夹
            breed_dir = os.path.join(multiclass_dir, breed)
            os.makedirs(breed_dir, exist_ok=True)
            shutil.copy(source_file, os.path.join(breed_dir, filename))
        elif breed[0].islower():  # 小写字母开头为狗
            shutil.copy(source_file, os.path.join(binary_dir, 'Dog', filename))
            # 同时复制到多分类目录中的对应品种文件夹
            breed_dir = os.path.join(multiclass_dir, breed)
            os.makedirs(breed_dir, exist_ok=True)
            shutil.copy(source_file, os.path.join(breed_dir, filename))

Mounted at /content/drive
images


In [5]:
#spliting datasets
import os
from sklearn.model_selection import train_test_split


def split_data(source_directory, train_dir, test_dir, split_size=0.2):
    categories = os.listdir(source_directory)
    print(f"Found categories: {categories}")  # 添加这行代码来打印分类列表
    for category in categories:
        category_path = os.path.join(source_directory, category)
        images = os.listdir(category_path)

        if not images:
            print(f"No images found in {category_path}. Skipping this category.")
            continue

        train_imgs, test_imgs = train_test_split(images, test_size=split_size, random_state=42)

        # 创建训练和测试目录
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # 复制图片到相应目录
        for img in train_imgs:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category, img))
        for img in test_imgs:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_dir, category, img))

# 使用函数切割数据
binary_source_dir = '/content/drive/My Drive/dogvscat/binary_classification'
binary_train_dir = '/content/drive/My Drive/dogvscat/train_binary'
binary_test_dir = '/content/drive/My Drive/dogvscat/test_binary'

multiclass_source_dir = '/content/drive/My Drive/dogvscat/multiclass_classification'
multiclass_train_dir = '/content/drive/My Drive/dogvscat/train_multiclass'
multiclass_test_dir = '/content/drive/My Drive/dogvscat/test_multiclass'

# 分割二分类数据集
split_data(binary_source_dir, binary_train_dir, binary_test_dir)

# 分割多类别数据集
split_data(multiclass_source_dir, multiclass_train_dir, multiclass_test_dir)


Found categories: ['Cat', 'Dog']
Found categories: ['beagle', 'Birman', 'Russian_Blue', 'Abyssinian', 'leonberger', 'miniature_pinscher', 'pug', 'american_bulldog', 'chihuahua', 'scottish_terrier', 'Ragdoll', 'yorkshire_terrier', 'great_pyrenees', 'keeshond', 'saint_bernard', 'newfoundland', 'havanese', 'wheaten_terrier', 'Egyptian_Mau', 'samoyed', 'japanese_chin', 'Maine_Coon', 'german_shorthaired', 'american_pit_bull_terrier', 'pomeranian', 'British_Shorthair', 'english_setter', 'boxer', 'Bengal', 'english_cocker_spaniel', 'Sphynx', 'Siamese', 'shiba_inu', 'staffordshire_bull_terrier', 'basset_hound', 'Bombay', 'Persian']


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.models as models
import torchvision
from torchvision.datasets import ImageFolder
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [9]:
#1.binary classification problem

device = torch.device("cuda:0")
model = models.resnet18(pretrained=True).to(device)


# 数据转换
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 数据集路径更新
dataset_path = '/content/drive/My Drive/dogvscat'
train_dataset_path = os.path.join(dataset_path, 'train_binary')
test_dataset_path = os.path.join(dataset_path, 'test_binary')

# Dataset and DataLoader
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pretrained ResNet18

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2).to(device)  # Adjusting the final layer for binary classification


# Loss and Optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = Adam(model.parameters(), lr=0.001)

# Training Function
def train_model(model, criterion, optimizer, num_epochs=10):
    model.train()  # 设置模型为训练模式
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            images = images.to(device)  # 把图像数据发送到GPU
            labels = labels.to(device)  # 把标签数据发送到GPU

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Training and Evaluation
train_model(model, criterion, optimizer)

def test_model():
    model.eval()  # 设置模型为评估模式
    total = correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)  # 把图像数据发送到GPU
            labels = labels.to(device)  # 把标签数据发送到GPU
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

test_model()

Epoch [1/10], Loss: 0.0800
Epoch [2/10], Loss: 0.0319
Epoch [3/10], Loss: 0.0710
Epoch [4/10], Loss: 0.0180
Epoch [5/10], Loss: 0.2121
Epoch [6/10], Loss: 0.0824
Epoch [7/10], Loss: 0.2129
Epoch [8/10], Loss: 0.0006
Epoch [9/10], Loss: 0.0012
Epoch [10/10], Loss: 0.0361
Test Accuracy: 94.64%


In [11]:
device = torch.device("cuda:0")
# 加载预训练模型并替换最后一层
model = models.resnet18(pretrained=True)
model = model.to(device)
for param in model.parameters():
    param.requires_grad = False  # 先冻结所有层

# 解冻一些层以进行微调
for param in model.layer4.parameters():
    param.requires_grad = True

# 替换最后的全连接层
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 37).to(device)

# 数据转换和增强
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and DataLoader
dataset_path = '/content/drive/My Drive/dogvscat'
train_dataset_path = os.path.join(dataset_path, 'train_multiclass')
test_dataset_path = os.path.join(dataset_path, 'test_multiclass')

# Dataset and DataLoader
train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 设置不同的学习率
optimizer = Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-4},  # 对于新解冻的层使用更高的学习率
    {'params': model.fc.parameters()}  # 默认学习率（1e-3）
], lr=1e-3)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss().to(device)



# Training and Evaluation
train_model(model, criterion, optimizer)
test_model()

Epoch [1/10], Loss: 1.0169
Epoch [2/10], Loss: 1.1364
Epoch [3/10], Loss: 0.2433
Epoch [4/10], Loss: 0.6332
Epoch [5/10], Loss: 0.8589
Epoch [6/10], Loss: 0.2649
Epoch [7/10], Loss: 0.6089
Epoch [8/10], Loss: 0.5384
Epoch [9/10], Loss: 0.1717
Epoch [10/10], Loss: 0.7013
Test Accuracy: 82.65%
